In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import pytz

# --------- TIME RESTRICTION (6 PM–9 PM IST) ----------
ist = pytz.timezone("Asia/Kolkata")
current_time = datetime.now(ist).time()

if not (current_time >= datetime.strptime("18:00","%H:%M").time() and
        current_time <= datetime.strptime("21:00","%H:%M").time()):
    print("Graph only visible between 6 PM and 9 PM IST")
    exit()

# --------- LOAD DATA ----------
df = pd.read_csv("test.csv")

# --------- FILTER CONDITIONS ----------
df = df[
    (df["Reviews"] > 500) &
    (~df["App"].str.startswith(("X","Y","Z"))) &
    (~df["App"].str.contains("S", case=False)) &
    (df["Category"].str.startswith(("E","C","B")))
]

# --------- TRANSLATE CATEGORY NAMES ----------
translations = {
    "Beauty": "सौंदर्य",      # Hindi
    "Business": "வணிகம்",     # Tamil
    "Dating": "Dating"        # will replace below in German
}

df["Category"] = df["Category"].replace(translations)
df["Category"] = df["Category"].replace({"Dating":"Partnersuche"})  # German

# --------- DATE PROCESSING ----------
df["Last Updated"] = pd.to_datetime(df["Last Updated"])
df = df.sort_values("Last Updated")

# --------- GROUP DATA ----------
grouped = df.groupby([pd.Grouper(key="Last Updated", freq="M"), "Category"])["Installs"].sum().reset_index()

# --------- PLOT ----------
plt.figure(figsize=(12,6))

for cat in grouped["Category"].unique():
    cat_df = grouped[grouped["Category"] == cat]
    
    plt.plot(cat_df["Last Updated"], cat_df["Installs"], label=cat)

    # ---- Highlight >20% MoM growth ----
    growth = cat_df["Installs"].pct_change()
    high_growth = growth > 0.20
    
    plt.fill_between(
        cat_df["Last Updated"],
        cat_df["Installs"],
        where=high_growth,
        alpha=0.3
    )

# --------- LABELS ----------
plt.title("Monthly Install Trends by Category")
plt.xlabel("Date")
plt.ylabel("Total Installs")
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()

plt.show()

PermissionError: [Errno 13] Permission denied: 'test.csv'